<a href="https://colab.research.google.com/github/ekaterinatao/NER_biomed_domain/blob/main/transformers_base/%D0%92%D0%9A%D0%A0_nerel_bio_RuBioRoBERTa_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Инструменты
Предобработанный дасасет [NEREL-BIO](https://huggingface.co/datasets/ekaterinatao/nerel_bio_ner_unnested)  
[Чек-пойнт](https://huggingface.co/ekaterinatao/nerel-bio-RuBioRoBERTa-base) дообученной модели на всем датасете nerel-bio  

Исходная модель [RuBioRoBERTa](https://huggingface.co/alexyalunin/RuBioRoBERTa)

### Установка зависимостей

In [ ]:
!pip install datasets accelerate evaluate wandb seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 k

In [ ]:
import numpy as np
import pandas as pd
import random
from dataclasses import dataclass

import torch
import datasets
from datasets import Dataset, DatasetDict
from transformers import (AutoTokenizer,
                          DataCollatorForTokenClassification,
                          RobertaForTokenClassification,
                          TrainingArguments, Trainer)
import evaluate

import warnings
warnings.filterwarnings("ignore")

In [ ]:
@dataclass
class TrainingConfig:
    seed = 64
    dataset = 'ekaterinatao/nerel_bio_ner_unnested'
    checkpoint = 'alexyalunin/RuBioRoBERTa'
    hf_repo_id = "ekaterinatao/nerel-bio-RuBioRoBERTa-base"
    n_labels = 45
    n_epochs = 10
    train_batch_size = 6
    eval_batch_size = 6
    device = "cuda" if torch.cuda.is_available() else "cpu"
    l_rate = 5e-05
    w_decay = 0.1
    warm_up = 0.1
    run_name = "RuBioRoBERTa-base"

config = TrainingConfig()

In [ ]:
seed = config.seed

random.seed(seed)
np.random.seed(seed)

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Скачивание датасета [NEREL-BIO](https://github.com/nerel-ds/NEREL-BIO)

In [ ]:
dataset = datasets.load_dataset(config.dataset)
dataset

Generating train split:   0%|          | 0/612 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/77 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/77 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'words', 'ner_tags'],
        num_rows: 612
    })
    valid: Dataset({
        features: ['id', 'words', 'ner_tags'],
        num_rows: 77
    })
    test: Dataset({
        features: ['id', 'words', 'ner_tags'],
        num_rows: 77
    })
})

In [ ]:
# Загрузка словаря токенов
url = 'https://raw.githubusercontent.com/ekaterinatao/NER_biomed_domain/main/labels.txt'
tags = pd.read_csv(url, names=['tag']).values.tolist()
tags = [item for sublist in tags for item in sublist]
tag_to_id = {tag: i for i, tag in enumerate(tags)}
id_to_tag = {i: tag for i, tag in enumerate(tags)}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.checkpoint, add_prefix_space=True, do_lower_case=True)

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.18M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["words"], truncation=True, max_length=512, is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_idxs = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_idxs:
            if word_idx is None:
                label_ids.append(-100) # Set the special tokens to -100.
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_dataset = dataset.map(
    tokenize_and_align_labels, batched=True,
    remove_columns = ['id', 'words', 'ner_tags']
)
tokenized_dataset

Map:   0%|          | 0/612 [00:00<?, ? examples/s]

Map:   0%|          | 0/77 [00:00<?, ? examples/s]

Map:   0%|          | 0/77 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 612
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 77
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 77
    })
})

# Обучение модели

In [ ]:
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [tags[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [tags[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
print(f'device is {config.device}')

device is cuda


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import os
os.environ["WANDB_PROJECT"]="ner_bert_nerel_bio"

In [ ]:
model = RobertaForTokenClassification.from_pretrained(
    config.checkpoint, num_labels=config.n_labels, id2label=id_to_tag, label2id=tag_to_id
)

training_args = TrainingArguments(
    output_dir=config.hf_repo_id,
    num_train_epochs=config.n_epochs,
    learning_rate=config.l_rate,
    weight_decay=config.w_decay,
    warmup_ratio=config.warm_up,
    per_device_train_batch_size=config.train_batch_size,
    per_device_eval_batch_size=config.eval_batch_size,
    group_by_length=True,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    evaluation_strategy="epoch",
    seed = config.seed,
    data_seed = config.seed,
    push_to_hub=True,
    save_strategy="no",
    report_to="wandb",
    run_name=config.run_name,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

config.json:   0%|          | 0.00/786 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at alexyalunin/RuBioRoBERTa and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


wandb: Currently logged in as: taoea. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.793199,0.412533,0.409423,0.410972,0.548362
2,No log,0.575139,0.771116,0.763486,0.767282,0.839158
3,No log,0.442625,0.805252,0.816254,0.810716,0.872699
4,No log,0.454533,0.807038,0.804947,0.805991,0.870671
5,0.866600,0.485428,0.809988,0.802356,0.806154,0.869267
6,0.866600,0.479122,0.819422,0.820966,0.820193,0.880499
7,0.866600,0.497499,0.820191,0.830624,0.825375,0.881591
8,0.866600,0.499688,0.821678,0.830389,0.826011,0.881747
9,0.866600,0.523686,0.823653,0.831802,0.827707,0.882059
10,0.054800,0.526243,0.825093,0.833451,0.829251,0.882683


TrainOutput(global_step=1020, training_loss=0.45209988849420174, metrics={'train_runtime': 448.2165, 'train_samples_per_second': 13.654, 'train_steps_per_second': 2.276, 'total_flos': 3669192307798524.0, 'train_loss': 0.45209988849420174, 'epoch': 10.0})

In [ ]:
wandb.finish()

eval/accuracy,▁▇████████
eval/f1,▁▇████████
eval/loss,█▂▁▁▁▁▁▁▁▁
eval/precision,▁▇████████
eval/recall,▁▇██▇█████
eval/runtime,▄█▁▁▁▅▁▁▂▁
eval/samples_per_second,▄▁███▄██▇█
eval/steps_per_second,▄▁███▄██▇█
train/epoch,▁▂▃▃▄▄▅▆▆▇███
train/global_step,▁▂▃▃▄▄▅▆▆▇███
train/grad_norm,█▁


In [ ]:
trainer.save_model(hf_repo_id)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

# Оценка качества на тестовой выборке

In [ ]:
model_fine = RobertaForTokenClassification.from_pretrained(
    config.hf_repo_id, num_labels=config.n_labels, id2label=id_to_tag, label2id=tag_to_id
)

config.json:   0%|          | 0.00/2.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [ ]:
def count_parameters(model: torch.nn.Module) -> int:
    """ Returns the number of learnable parameters for a PyTorch model """
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model_fine)

354356269

In [ ]:
training_args = TrainingArguments(
    output_dir='./',
    num_train_epochs=config.n_epochs,
    learning_rate=config.l_rate,
    weight_decay=config.w_decay,
    warmup_ratio=config.warm_up,
    per_device_train_batch_size=config.train_batch_size,
    per_device_eval_batch_size=config.eval_batch_size,
    group_by_length=True,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    evaluation_strategy="epoch",
    seed = config.seed,
    data_seed = config.seed
)

trainer_fine = Trainer(
    model=model_fine,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
preds, labels, metrics = trainer_fine.predict(test_dataset=tokenized_dataset["test"])

In [ ]:
metrics

{'test_loss': 0.5129269361495972,
 'test_precision': 0.8504919606431486,
 'test_recall': 0.8494726749760306,
 'test_f1': 0.8499820122316825,
 'test_accuracy': 0.9040091264667536,
 'test_runtime': 3.221,
 'test_samples_per_second': 23.905,
 'test_steps_per_second': 4.036}

In [ ]:
predictions = np.argmax(preds, axis=2)
tags_mod = [f'_{t}' for t in tags]

true_predictions = [
    [tags_mod[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [tags_mod[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = seqeval.compute(predictions=true_predictions, references=true_labels)

del results['overall_precision']
del results['overall_recall']
del results['overall_f1']
del results['overall_accuracy']

In [ ]:
df = pd.DataFrame.from_dict(results).transpose().sort_values(by='f1', ascending=False).iloc[:, :3]
df['f1'] = df['f1'].apply(lambda x: round(x * 100, 2))
df['precision'] = df['precision'].apply(lambda x: round(x * 100, 2))
df['recall'] = df['recall'].apply(lambda x: round(x * 100, 2))
df

,precision,recall,f1
MONEY,100.00,100.00,100.00
PERCENT,97.26,98.61,97.93
PROFESSION,97.14,94.44,95.77
DATE,93.12,96.75,94.90
LIVB,94.94,93.75,94.34
NUMBER,92.50,91.36,91.93
PERSON,92.25,90.84,91.54
CHEM,87.88,90.22,89.04
TIME,100.00,80.00,88.89
DISO,88.52,87.82,88.17
